In [1]:
from wyborcza import get_article_data
import pandas as pd
from selenium import webdriver
from tqdm import tqdm
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--headless")
chrome_options.add_argument("user-data-dir=selenium")

In [2]:
df = pd.read_csv("wyborcza.csv")
df.head()

,Title,Department,Author,Date,Text,Link
0,BO ZABRAKŁO JAROSŁAWA,Gazeta Wyborcza,TOMASZ URZYKOWSKI,2019-01-02,"Przygotowana z rozmachem ekspozycja ""Znaki wol...",https://classic.wyborcza.pl/archiwumGW/9012194...
1,TAK W PAŃSTWIE PiS FAŁSZUJĄ HISTORIĘ,Gazeta Wyborcza,NaN,2019-01-02,To zdjęcie przedstawia Lecha i Jarosława Kaczy...,https://classic.wyborcza.pl/archiwumGW/9012193...
2,SCENARIUSZE NA NOWY ROK,Gazeta Wyborcza,WITOLD GADOMSKI,2019-01-02,"To będzie rok wyborczy, więc działania polityk...",https://classic.wyborcza.pl/archiwumGW/9012195...
3,KTO POKIERUJE UNIĄ EUROPEJSKĄ,Gazeta Wyborcza,BARTOSZ T. WIELIŃSKI,2019-01-02,Oficjalnie Angela Merkel o przeprowadzce do br...,https://classic.wyborcza.pl/archiwumGW/9012198...
4,"NBP pisze do ""Wyborczej""",Gazeta Wyborcza,"AGNIESZKA KUBLIK, WOJCIECH CZUCHNOWSKI",2019-01-02,"Pytania do NBP, m.in. o związki prezesa Adama ...",https://classic.wyborcza.pl/archiwumGW/9012196...


In [3]:
df.tail()

,Title,Department,Author,Date,Text,Link
204320,"AGA ZANO, TŁUMACZKA",NaN,ARKADIUSZ GRUSZCZYŃSKI,2022-12-31,Aga Zano to tłumaczka z języka angielskiego ta...,https://classic.wyborcza.pl/archiwumGW/9231541...
204321,"MAGDA SZPECH, AKTYWISTKA, CYBERELFKA",NaN,ARKADIUSZ GRUSZCZYŃSKI,2022-12-31,"Szpecht jest reżyserką teatralną, która 24 lut...",https://classic.wyborcza.pl/archiwumGW/9231545...
204322,"ANNA BAŃKOWSKA, TŁUMACZKA",NaN,MAGDALENA KARST-ADAMCZYK,2022-12-31,Nowe tłumaczenie książki znanej przez dekady p...,https://classic.wyborcza.pl/archiwumGW/9231553...
204323,"KRYSTYNA LEWENFISZ, PRODUCENTKA",NaN,PAULA SZEWCZYK,2022-12-31,"Gdy przeczytała książkę ""Mama zawsze wraca"", p...",https://classic.wyborcza.pl/archiwumGW/9231554...
204324,"DR HANNA MACHIŃSKA, PRAWNICZKA, ZASTĘPCZYNI RPO",NaN,EDYTA BRYŁA,2022-12-31,"""Jako kobieta byłam bardzo wrażliwa na różne s...",https://classic.wyborcza.pl/archiwumGW/9231555...


In [4]:
links = df[df['Title'] == 'ERROR'].Link
driver = webdriver.Chrome(options=chrome_options)
data_list = []
for link in tqdm(links):
    data_list.append(get_article_data(link, driver))

0it [00:00, ?it/s]


In [5]:
df_corrected = pd.DataFrame(data_list)
df_corrected = df_corrected.rename(columns=lambda x: x[0].upper() + x[1:])

In [9]:
df_corrected.head()

""


In [8]:
df_prime = df.set_index("Link")
df_prime.update(df_corrected.set_index("Link"))

KeyError: "None of ['Link'] are in the columns"

In [6]:
df_prime.to_csv("wyborcza_2019-2023.csv")